In [ ]:
import cv2
from pydub.generators import Sine
from pydub import AudioSegment
import numpy as np
from tqdm import tqdm  # Importing tqdm for progress tracking

def pixel_to_sound(image, tone_duration=50, block_size=16, max_duration=15000):
    height, width, _ = image.shape
    sound = AudioSegment.silent(duration=0)  # Start with silent audio
    total_tones = max_duration // tone_duration  # Max number of tones

    # Downsample image by processing blocks of pixels
    row_step = max(1, height // total_tones)

    for i in tqdm(range(0, height, row_step), desc="Processing rows", unit="row"):
        for j in range(0, width, block_size):
            block = image[i:i+row_step, j:j+block_size]
            average_pixel = np.mean(block, axis=(0, 1))  # Taking Average of RGB values
            brightness = int(np.mean(average_pixel))
            frequency = 200 + (brightness / 255) * 1800  # Scale brightness to frequency

            # A sine wave at the mapped frequency
            tone = Sine(frequency).to_audio_segment(duration=tone_duration)

            # Adding tone to the overall sound
            sound += tone

            # Stop if we exceed max_duration
            if len(sound) >= max_duration:
                break

    return sound[:max_duration]  # Trim the sound to max_duration

image_path = "image_used.png"
image = cv2.imread(image_path)  
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

# Downsample the image to speed up processing and reduce sound length
image = cv2.resize(image, (100, 100))  # Resize to 100x100 pixels

# Generate sound from the image with progress tracking
print("Generating sound from the image...")
sound = pixel_to_sound(image)

# Export the sound to a file
sound.export("image_sound_short.wav", format="wav")
print("Sound generated and saved as 'image_sound.wav'.")


Generating sound from the image...


Processing rows: 100%|█████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 186.09row/s]

Sound generated and saved as 'image_sound.wav'.
